The Probabilistic Sharpe Ratio (PSR) takes into account the kurtosis and skewness of returns, in addition to just the mean and standard deviation. In practical terms, this helps in scertaining confidence bands and statistical significance of the Sharpe Ratio itself, and express it in probablistic terms as the name implies (eg: a 95% chnace of Sharpe Ratio being 1.5)

Formula:

Given :-  

  - a predefined benchmark Sharpe Ratio (SR*) {this can be set to zero}
  - an estimated Sharpe Ratio (SRˆ)

The PSR can be expressed in probabilistic terms as:

PSR(SR*) = Prob[SR <= SRˆ]


----------
THIS CAN BE RE-WRITTEN AS:

PSR(SR*) = Z[(SR^- SR*) / sigmaˆ(SRˆ)]

WHERE:

 - The above function is a cumulative distribution function (CDF)
 - Z is the CDF of the standard normal distribution

In [14]:
import pandas as pd
import scipy.stats as ss
apple = pd.read_csv(r"/content/drive/MyDrive/AAPL.csv", index_col = "Date", parse_dates = True)

In [15]:
apple["daily returns"] = apple["Adj Close"].pct_change()
apple = apple.dropna()

In [16]:
apple.head()

,Open,High,Low,Close,Adj Close,Volume,daily returns
Date,,,,,,,
2016-09-07,26.957500,27.190001,26.767500,27.090000,25.406099,169457200,0.006128
2016-09-08,26.812500,26.817499,26.309999,26.379999,24.740232,212008000,-0.026209
2016-09-09,26.160000,26.430000,25.782499,25.782499,24.179869,186228000,-0.022650
2016-09-12,25.662500,26.430000,25.632500,26.360001,24.721476,181171200,0.022399
2016-09-13,26.877501,27.197500,26.809999,26.987499,25.309967,248704800,0.023805


In [35]:
def calc_daily_sharpe_ratio(df, rf_rate = 0):
  
  mean_return = df["daily returns"].mean()
  std_returns = df["daily returns"].std()

  sharpe_ratio = (mean_return - rf_rate) / std_returns
  sharpe_ratio *= 252**0.5
  
  return sharpe_ratio

In [36]:
calc_sharpe_ratio(apple)

1.3494385329110765

In [41]:
def calc_PSR(df, benchmark_sr = 0):

  sr = calc_sharpe_ratio(df)
  skew = ss.skew(df["daily returns"])
  kurtosis = ss.kurtosis(df["daily returns"], fisher = True)

  n = len(df)

  sigma_sr = ((1/(n-1)) * (1 + 0.5*sr**2 + skew*sr+(kurtosis/4)*sr**2))**0.5

  ratio = (sr - benchmark_sr) / sigma_sr
  psr = ss.norm.cdf(ratio)
  annual_psr = psr * (252**0.5)

  print(f"PSR : {psr}")

  return psr


In [42]:
calc_PSR(apple)

PSR : 1.0


1.0